# 卷积神经网络

卷积神经网络中的输入和输出有些区别, 需要重新设计, 训练模块基本上是一致的


In [1]:
# imports
import torch
from torchvision import datasets, transforms
from torch import optim


## 先读取数据

- 分别创建训练集和测试集
- datasetloader 来进行迭代数据


In [2]:
# 定义超参数
input_size = 28  # 图像的像素为28*28*1 注意, 这里是3维的数据
num_classes = 10  # 标签的种类, 也就是最终分类的数量
num_epoches = 3  # 训练循环周期, 总共训练3次
batch_size = 64  # 一个批次为64张图片

# 下载训练集
train_dataset = datasets.MNIST(
    root='../../data/mnist2/',  # 数据下载的位置
    train=True,  # 默认就是true, 也就是下载训练集训练集的文件是train-images-idx3-ubyte
    # 使用transform包下面的ToTensor转换所有的数据为tensor的形式, 也就是封装数据, 注意这里需要方法的返回值, 而不是传入方法
    transform=transforms.ToTensor(),
    download=True  # 下载到本地, 可以直接使用网络流进行训练
)
print("训练集下载完毕")

# 下载测试集
test_dataset = datasets.MNIST(
    root='../../data/mnist2/',
    train=False,  # 下载测试集, 测试集的文件名为 t10k-images-idx3-ubyte,
    download=True,
    transform=transforms.ToTensor()  # 转换为Tensor格式
)
print("测试集下载完毕")

# 将训练集和测试集封装进入loader中
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,  # 必须是map类型或者iterable类型, MNIST实现了map所需要的方法
    batch_size=batch_size,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)


训练集下载完毕
测试集下载完毕


## 卷积神经网络模块构建

- 一般卷积层, relu 层, 池化层可以写成一个套餐, 直接加入进来进行操作. 这样可以看做是模块化的, 因为 relu 和池化层都不能算是一个计算的层, 而是一个加单的加减.
- 注意卷积最后结果还是一个特征图, 需要把图转换成向量才能做分类或者回归任务.

这里, 计算卷积特征图的公式为：

长度: $h_2=\frac{H_1-F_H-2P}{S} + 1$
宽度: $W_2=\frac{W_1-F_W-2P}{S} + 1$

这里$h_1,w_1$表示的是特征图的长宽, P 表示的是 padding 的大小, S 表示的是卷积核的步长


In [3]:
# 建立模型
import torch.nn as nn

"""
这里引入nn模块, 在建立class的时候继承的是nn.Module的模块
"""


class CNN(nn.Module):
    # 定义一个网络模型, 网络模型的名字为CNN, 也就是这个类
    def __init__(self) -> None:
        # 调用父类的方法, 将所有的方法继承过来
        # 这里调用一下父类的初始化方法, 初始化方法帮我们初始化所有的参数. 也可以使用下面的来赋予self到初始化方法中
        super().__init__()
        # super(CNN, self).__init__()

        # 第一个卷积模块, 如上面所说, 这个模块包含了relu和池化层
        # Sequential模块包含了多个参数, 每一个参数都是一个操作, 比如：
        #   >>> model = nn.Sequential(nn.Conv2d(1, 20, 5),nn.ReLU(),nn.Conv2d(20, 64, 5),nn.ReLU())
        # 上面就创造了一个2个卷积层和2个relu层的网络结构
        #
        # 第一个二维卷积层, 根据公式得到的大小：
        #   ((28 - 5 + 2 * 2） / 1） + 1 = 28
        # 因此第一个卷积层的二维卷积核得到的每一个特征图的大小为28*28大小是不变的
        # 经过relu层是不变的
        # 但是经过池化层 由于我们使用的是长度为2的核, 因此应该会得到一个 14*14的特征图
        # 我们总共需要输出16个图片, 因此会有16个卷积核, 因此第一个模块我们的输出就是14*14*16
        self.conv1 = nn.Sequential(
            nn.Conv2d(  # 创建一个二维的卷积层
                in_channels=1,  # 输入一个特征图
                out_channels=16,  # 输出16个特征图, 也就是有16个卷积核
                kernel_size=5,  # 每一个卷积核的大小为5, 也就是5x5的卷积核
                stride=1,  # 卷积核每次移动的的步长为1
                padding=2  # 图片外侧padding2个格
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)  # 生成一个max池化层, 核长度为2x2
        )

        # 第二个卷积模块, 输入16个特征图, 输出32个特征图.
        # 我们输入的特征图为 14*14*16 (我们上面得到的)
        # 第二层我们一样使用长度为5的核, 因此
        #   (14 - 5 + 4)/1 + 1 = 16
        # 我们得到的结果保持不变, 特征图为14*14的图片
        # 经过relu层, 特征不变
        # 经过池化层, 因为我们使用了长度为2的核, 因此 14/2 = 7, 我们所有的特征图都变成了7*7的特征图
        # 而我们总共需要输出32个图片, 因此7*7*32
        self.conv2 = nn.Sequential(
            # 16个特征图输入, 32个特征图输出, 5x5的卷积核, 1个步长, 外部padding2个格
            # 也可以使用较为复杂的卷积核设置：
            #   >>> m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # 创建2层的全链接层, 第一层的数量为32 * 7 * 7, 第二层的数量为10, 也就是输出层
        self.out = nn.Linear(32 * 7 * 7, 10)

    # 前向传播, 这里的输入x就是一张图片的数据, 也就是一个Tensor矩阵
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # 这里对PyTorch的Tensor做了一个reshape的操作, view方法就是维度变换, 这里将所有的大小直接拉伸为1维的向量
        #
        # 比如：x = torch.tensor([1,2,3,4])
        # 我们想把x拉伸成一个2x2的矩阵我们可以用 x.view(2,2) 但是第二个参数我们没有必要算的,
        # 因为如果第一个参数固定了, 第二个参数必然是一个定值, 我们可以让pytorch进行计算, 可以使用-1
        # 因此可以使用 x.view(2,-1) 或者 x.view(-1,2) 效果是一样的
        #
        # 同样size方法也是将tensor自带的属性打印出来, 这里的参数是维度的意思
        # t = torch.empty(3, 4, 5)
        # t.size() # 结果是tensor.size(3). 此时我们给size加上一个获取第0个参数, 我们就可以得到3
        # t.size(0) # 结果就是3, 因为我们获取的是tensor中第0个数据, 也就是3.
        # 如果维度更高的话我们获取的就是不同维度的size, 但是这里的size返回结果是一个单维度的数据, 用法就很简单.
        #
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output


## 准确率的评估函数

使用准确率来计算预测值和结果值复合的值是多少


In [4]:
# 创建一个方法来计算当前模型的准确率
# 在这里调用了一个max方法, 这个max方法可以将Tensor矩阵的最大值求出来
# 如果直接使用 torch.max(torch.Tensor) 方法返回一个tensor, 里面包含着最大值 troch.tensor(12345)
#   这个值是整个矩阵的最大值, 无论行列
# 如果加上一个参数的话就是获取行或者列的最大值, 0 就是列, 1 就是行
# 下面的就是获得所有列的最大值, 返回一个tensor, 返回的tensor是一个array
#   torch.max(predictions.data, 0)
#   返回的是 torch.return_types.max([[1,2,3,2],[0,1,1,1]]) 第一个参数是数值, 第二个参数是index
# 如果需要获得所有行的值就用参数1, torch.max(data, 1), 然后获取index的位置就是torch.max(data, 1)[1]
def accuracy(predictions, labels):
    # 获得每一行最大值的index
    pred_index = torch.max(predictions.data, 1)[1]
    rights = pred_index.eq(labels.data.view_as(pred_index)).sum()
    return rights, len(labels)


## 开始训练！

我们使用常用的交叉熵损失函数 CrossEntropyLoss(), 什么是交叉熵？

交叉熵主要是用来判定实际的输出与期望的输出的接近程度,为什么这么说呢,举个例子：在做分类的训练的时候,如果一个样本属于第 K 类,那么这个类别所对应的的输出节点的输出值应该为 1,而其他节点的输出都为 0,即[0,0,1,0,….0,0],这个数组也就是样本的 Label,是神经网络最期望的输出结果。也就是说用它来衡量网络的输出与标签的差异,利用这种差异经过反向传播去更新网络参数。

举个例子：假如小明和小王去打靶,那么打靶结果其实是一个 0-1 分布,X 的取值有{0：打中,1：打不中}。在打靶之前我们知道小明和小王打中的先验概率为 10%,99.9%。根据上面的信息量的介绍,我们可以分别得到小明和小王打靶打中的信息量。但是如果我们想进一步度量小明打靶结果的不确定度,这就需要用到熵的概念了。那么如何度量呢,那就要采用期望了。我们对所有可能事件所带来的信息量求期望,其结果就能衡量小明打靶的不确定度.

因此也是计算二分类问题的很好损失函数.

对优化器的详解：
<https://blog.csdn.net/qq_39852676/article/details/105919329>


In [5]:
# 将我们刚刚定义的网络模型拿过来, 创建一个实体类
nnet = CNN()

# 损失函数, 损失函数会实现一个forward(data,targetForCompare)的方法, 可以计算损失值
criterion = nn.CrossEntropyLoss()

r"""
优化器
优化算法的功能,是通过改善训练方式,来最小化(或最大化)损失函数E(x)。自适应算法能很快收敛,并快速找到参数更新中正确的目标方向; 而标准的SGD、NAG和动量项等方法收敛缓慢,且很难找到正确的方向。

torch.optim是一个实现了各种优化算法的库。大部分常用的方法得到支持,并且接口具备足够的通用性,使得未来能够集成更加复杂的方法
为了使用torch.optim,你需要构建一个optimizer对象。这个对象能够保持当前参数状态并基于计算得到的梯度进行参数更新。为了构建一个Optimizer,你需要给它一个包含了需要优化的参数(必须都是Variable对象)的iterable。然后,你可以设置optimizer的参
数选项,比如学习率,权重衰减,等等。
Optimizer也支持为每个参数单独设置选项。若想这么做,不要直接传入Variable的iterable,而是传入dict的iterable。每一个dict都分别定
义了一组参数,并且包含一个param键,这个键对应参数的列表。其他的键应该optimizer所接受的其他参数的关键字相匹配,并且会被用于对这组参数的
"""
optimizer = optim.Adam(nnet.parameters(), lr=0.001)  # 我们定义一个普通的随机梯度下降算法的优化器

r"""
开始训练：
我们使用一个for循环来进行训练
"""
for epoch in range(num_epoches):
    train_rights = []  # 当前的epoch的结果保存下来的容器

    # 这里用的enumerate(sequence, [start=0])就是直接将sequence序列的迭代出来.
    #   for i, element in enumerate(seq): # 这里就可以得到index和当前迭代的数值
    # 在迭代出来的结果中, 我们直接在方法中将data和target结构出来
    # 验证网络处于 train 或 eval 模式,其最后结果是不一样的
    # train模式启用 BatchNormalization 和 Dropout, 而eval则不启用
    for index, (data, target) in enumerate(train_loader):
        # 这里调用一下train方法, 我们将模型设定为training形态. 等同于nnet.mode(True)
        # 除了train模式还有eval模式
        # 我们通常要在batch的for循环内部启用train模式, 而不是在外部.
        # 因为如果我们在训练过程中调用了test函数,我们就会进eval模式,直到下一次train函数被调用。
        # 这就导致了每一个epoch中只有一个batch使用了dropout ,这就导致了我们看到的性能下降。
        nnet.train()

        # 传入数据
        output = nnet(data)

        # 将输出数据和目标值输入进去计算一次损失值
        loss = criterion(output, target)

        # 建梯度清零
        # 反向传播以前都需要将梯度清零, 因为如果不清零pytorch会将上次计算的梯度和本次计算的梯度进行累加
        optimizer.zero_grad()

        # 对于当前的图片结束以后进行反向传播, 求导计算
        loss.backward()

        # 将优化其中的权重参数全部更新, 更新权重参数, 准备进行重新计算
        # 所有的optimizer都实现了step()方法, 这个方法会更新所有的参数。
        optimizer.step()

        # 将当前准确率放入全局容器中, 用来展示, 可以删除
        train_right = accuracy(output, target)  # 返回的是一个tuple, (准确个数, 总体个数)
        train_rights.append(train_right)

        if index % 100 == 0:  # 展示准确率
            # 设置当前模式为eval模式, 也就是计算模式
            nnet.eval()
            val_rights = []
            # 将测试机的数据进行输出一遍, 查看错误率
            for (data, target) in test_loader:
                output = nnet(data)
                right = accuracy(output, target)  # 得到错误率, 返回总共多少正确的和总体长度
                val_rights.append(right)  # 将当前的测试集正确率放到容器中
            # 计算出当前训练集的准确率
            train_rate = (sum([rate_pair[0] for rate_pair in train_rights]),
                          sum([rate_pair[1] for rate_pair in train_rights]))
            # 计算出测试集的准确率
            val_rate = (sum([rate_pair[0] for rate_pair in val_rights]),
                        sum([rate_pair[1] for rate_pair in val_rights]))
            print("当前epoch: {} [{}/{} ({:.0f}%)] 损失: {:.6f} | 训练集准确率：{:.2f}% 测试集正确率: {:.2f}%".format(
                epoch, index * batch_size, len(train_loader.dataset),
                100. * index / len(train_loader),
                loss.data,
                100. * train_rate[0].numpy() / train_rate[1],
                100. * val_rate[0].numpy() / val_rate[1]
            ))


当前epoch: 0 [0/60000 (0%)] 损失: 2.296350 | 训练集准确率：6.25% 测试集正确率: 18.33%
当前epoch: 0 [6400/60000 (11%)] 损失: 0.369493 | 训练集准确率：74.15% 测试集正确率: 91.34%
当前epoch: 0 [12800/60000 (21%)] 损失: 0.140937 | 训练集准确率：83.64% 测试集正确率: 95.38%
当前epoch: 0 [19200/60000 (32%)] 损失: 0.147676 | 训练集准确率：87.66% 测试集正确率: 96.62%
当前epoch: 0 [25600/60000 (43%)] 损失: 0.066431 | 训练集准确率：89.71% 测试集正确率: 96.98%
当前epoch: 0 [32000/60000 (53%)] 损失: 0.058125 | 训练集准确率：91.11% 测试集正确率: 97.29%
当前epoch: 0 [38400/60000 (64%)] 损失: 0.110508 | 训练集准确率：92.10% 测试集正确率: 97.80%
当前epoch: 0 [44800/60000 (75%)] 损失: 0.122578 | 训练集准确率：92.84% 测试集正确率: 97.88%
当前epoch: 0 [51200/60000 (85%)] 损失: 0.052326 | 训练集准确率：93.42% 测试集正确率: 97.70%
当前epoch: 0 [57600/60000 (96%)] 损失: 0.088413 | 训练集准确率：93.85% 测试集正确率: 97.94%
当前epoch: 1 [0/60000 (0%)] 损失: 0.060140 | 训练集准确率：98.44% 测试集正确率: 98.19%
当前epoch: 1 [6400/60000 (11%)] 损失: 0.050169 | 训练集准确率：97.85% 测试集正确率: 98.13%
当前epoch: 1 [12800/60000 (21%)] 损失: 0.075982 | 训练集准确率：97.94% 测试集正确率: 98.43%
当前epoch: 1 [19200/60000 (32%)] 损失: 0.0